In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wordninja
!pip install pdfplumber
!pip install python-docx
import nltk
import spacy
import re
import requests
import subprocess
import pandas as pd
nltk.download('maxent_ne_chunker_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

nlp = spacy.load('en_core_web_sm')

import pdfplumber

from spacy.matcher import Matcher
from docx import Document
from bs4 import BeautifulSoup

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=ad0bb24b4af61161b2534cd627a72c7c1e1df69749a8707787989b06049e18ad
  Stored in directory: /root/.cache/pip/wheels/e6/66/9c/712044a983337f5d44f90abcd244bd4b8ad28ee64750404b50
Successfully built wordninja
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.0 MB/s eta 0:00:00


[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/

In [ ]:
import os
resume_folder="/content/drive/MyDrive/resume"
resume_files = [os.path.join(resume_folder, f) for f in os.listdir(resume_folder)]

print(f"Total resumes found: {len(resume_files)}")
resume_files

Total resumes found: 11


['/content/drive/MyDrive/resume/cv - Nes.pdf',
 '/content/drive/MyDrive/resume/ahle-cv.pdf',
 '/content/drive/MyDrive/resume/inbound3334998641584266252 - Riahi Firas.pdf',
 '/content/drive/MyDrive/resume/cv-moem.pdf',
 '/content/drive/MyDrive/resume/cv_aziz_.pdf',
 '/content/drive/MyDrive/resume/zeineb_elyounsi_cv.pdf',
 '/content/drive/MyDrive/resume/el_younsi_yosr_devops_engineer.pdf',
 '/content/drive/MyDrive/resume/Cv_arifi_amine_anglais.pdf',
 "/content/drive/MyDrive/resume/Nour's Resumee.pdf",
 '/content/drive/MyDrive/resume/cv_mod.pdf',
 '/content/drive/MyDrive/resume/Software Engineer (4).pdf']

Extraction du texte

In [ ]:


def extract_text(file_path):
    if file_path.endswith('.pdf'):
        with pdfplumber.open(file_path) as pdf:
            # Ensure to check that page.extract_text() is not None before joining
            return "\n".join(page.extract_text() or '' for page in pdf.pages)
    elif file_path.endswith('.docx'):
        doc = Document(file_path)
        return "\n".join(para.text for para in doc.paragraphs)
    elif file_path.endswith('.txt') or file_path.endswith('.md'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    elif file_path.endswith('.html'):
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        # Parse the HTML and extract text
        soup = BeautifulSoup(content, 'html.parser')
        return soup.get_text(separator='\n')
    else:
        raise ValueError(f"Unsupported file type: {file_path}")
extraction={}
all_resumes_text ={}
for resume in resume_files:
    all_resumes_text[resume]=extract_text(resume)

print("Extraction complete for all resumes!")


Extraction complete for all resumes!


Extraction d'info

In [ ]:

def extract_contact_info(text):
    """Extract contact information using regex"""
    email = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)
    phone_ = re.findall(r'\+?\d{1,3}(?:[ \-\*\.]?\d{2,4}){2,4}', text)
    phone_ = re.findall(r'\+?\d{1,3}(?:[ \-\*\.]?\d{1,4}){2,4}', text)
    phone=[p for p in phone_ if len(p)>=7]
    return {
        'email': email[0] if email else None,
        'phone': phone[0] if phone else None
    }

def extract_names(text):

    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            if  re.search(r'(email|e-mail|linkedin)',ent.text.lower(), re.IGNORECASE):
                return " ".join(text[:30].split()[:2])
            return ent.text.strip()
    return ""


for resume in resume_files:

 f=extract_contact_info(all_resumes_text[resume])
 p=extract_names(all_resumes_text[resume])
 extraction[resume]={"name":p,"email":f["email"],"phone":f["phone"]}
print(extraction)


{'/content/drive/MyDrive/resume/cv - Nes.pdf': {'name': 'Nesrine Khiyari', 'email': 'nesrine.khiyari@etudiant-enit.utm.tn', 'phone': '+216 56 42 57 04'}, '/content/drive/MyDrive/resume/ahle-cv.pdf': {'name': 'Ahlem BRINSI', 'email': 'ahlem.brinsi@etudiant-enit.utm.tn', 'phone': '+21628248090'}, '/content/drive/MyDrive/resume/inbound3334998641584266252 - Riahi Firas.pdf': {'name': 'firas.riahi@ensi-uma.tn', 'email': 'firas.riahi@ensi-uma.tn', 'phone': '50 385 961'}, '/content/drive/MyDrive/resume/cv-moem.pdf': {'name': 'BEN', 'email': 'moemen.benaissa@etudiant-enit.utm.tn', 'phone': '+21650182149'}, '/content/drive/MyDrive/resume/cv_aziz_.pdf': {'name': 'Mohamed Aziz ben kileni', 'email': 'mohamedaziz.benkileni@etudiant-enit.utm.tn', 'phone': '+21620537433'}, '/content/drive/MyDrive/resume/zeineb_elyounsi_cv.pdf': {'name': 'Zeineb El Younsi', 'email': 'younsizeineb45@gmail.com', 'phone': '+216 29 425 399'}, '/content/drive/MyDrive/resume/el_younsi_yosr_devops_engineer.pdf': {'name': 'Yo



Texte propre



In [ ]:
import wordninja
import re


def fix_concatenated_words(text, threshold=15):

    # Split text into tokens; \w+ matches words, [^\w\s] matches punctuation
    tokens = re.findall(r'\w+|[^\w\s]', text, re.UNICODE)
    new_tokens = []
    for token in tokens:
        # If token is long and alphabetic (likely a concatenated word), split it
        if len(token) > threshold and token.isalpha():
            splitted = wordninja.split(token)
            # Join the split parts with a space and add them as one token
            new_tokens.append(" ".join(splitted))
        else:
            new_tokens.append(token)
    # Reassemble tokens into a string and normalize spaces
    result = " ".join(new_tokens)
    #result = re.sub(r'\s+', ' ', result)
    return result

# Example usage:
def fix_concatenated_words_line(line, threshold=15):
    # Split the line into tokens (keeping punctuation intact)
    tokens = re.findall(r'\w+|[^\w\s]', line, re.UNICODE)
    new_tokens = []
    for token in tokens:
        # If token is long and alphabetic, split it
        if len(token) > threshold and token.isalpha():
            splitted = wordninja.split(token)
            new_tokens.append(" ".join(splitted))
        else:
            new_tokens.append(token)
    # Reassemble tokens into a string (preserving the line's newline later)
    result_line = " ".join(new_tokens)
    result_line = re.sub(r'[ \t]+', ' ', result_line)  # Normalize spaces only
    return result_line

def fix_concatenated_words(text, threshold=15):
    # Process the text line by line to preserve newlines
    lines = text.split('\n')
    fixed_lines = [fix_concatenated_words_line(line, threshold) for line in lines]
    return "\n ".join(fixed_lines)


def clean_text(text):


    # Remove headers/footers
    text = re.sub(r'\b(Page \d+|Resume|CV|Confidential|©.*|Date:.*)\b', '', text, flags=re.I)

    # Standardize section headers with comprehensive mapping
    section_mapping={
    # Experience
    r'\n\s*[\W]*?(Professional (Experience|Background)|Work (History|Experience|Exp\.?)|Career (History|Summary|Experience)|Employment( History)?|Positions (Held|Worked)|Job (History|Experience|Roles)|Relevant Experience|EXPERIENCE)': 'EXPERIENCE',

    # Education
    r'\n\s*[\W]*?(Academic (Background|Qualifications|Record|Training)|Education(al)? (Background|History|Qualifications|Achievements|Details)|Degrees|Schools (Attended|Graduated From)|University|College|Certifications & Education|Diplomas and Training|EDUCATION)': 'EDUCATION',

    # Skills
    r'\n\s*[\W]*?(Technical (Skills|Expertise|Abilities|Proficiency|Knowledge)|Core Competencies|Key Skills|Areas of Expertise|Professional Skills|Skill Set|Abilities & Expertise|Hard Skills|Soft Skills|Qualifications|Skills Summary|SKILLS|SKILLS & LANGUAGES)': 'SKILLS',

    # Certifications
    r'\n\s*[\W]*?(Certifications?|Licenses?|Credentials|Professional Development|Accreditations|Training & Certifications|Certifications & Licenses|Professional Certifications|CERTIFICATIONS)': 'CERTIFICATIONS',

    # Projects
    r'\n\s*[\W]*?(Key Projects|Notable Projects|Project (Experience|History|Portfolio|Work)|Selected Projects|Major Projects|Freelance Work|Personal Projects|Technical Projects|PROJECTS)': 'PROJECTS',

    # Achievements / Awards
    r'\n\s*[\W]*?(Honors & Awards|Achievements|Recognitions?|Awards|Accomplishments|Distinctions|Honors & Distinctions|AWARDS)': 'AWARDS',

    # Publications / Research
    r'\n\s*[\W]*?(Publications?|Research (Work|Papers|Projects|Experience)|Journal Articles|Conference Papers|Academic Publications|Published Work|Scientific Contributions|PUBLICATIONS)': 'PUBLICATIONS',

    # Volunteering / Extracurricular Activities
    r'\n\s*[\W]*?(Volunteering (Experience|Work)?|Community (Service|Involvement)|Extracurricular (Activities|Engagements)|Social Contributions|Leadership Experience|VOLUNTEER EXPERIENCE|Volunteer Experience|VOLUNTEERING)': 'VOLUNTEERING',

    # Languages
    r'\n\s*[\W]*?(Languages?|Spoken Languages|Language Proficiency|Language Skills|LANGUAGES|Languages)': 'LANGUAGES',

    # Interests / Hobbies
    r'\n\s*[\W]*?(Hobbies & Interests|Interests?|Personal Interests|Extracurricular Interests|Passions|Centers of Interest|Hobbies|INTERESTS)': 'INTERESTS',

    # References
    r'\n\s*[\W]*?(References?|Professional References|Referees|Endorsements|REFERENCES)': 'REFERENCES'
}

     # Normalize line breaks
    text = text.replace('\r\n', '\n')
    # Reduce multiple newlines to two
    text = re.sub(r'\n{3,}', '\n\n', text)
    # Remove extra spaces or tabs
    text = re.sub(r'[ \t]{2,}', ' ', text)
    # Replace various bullet characters with a space
    #text = re.sub(r'[•●♦▪➢]', ' ', text)
    # Remove special characters except allowed ones (@, ., ,, (), and -)
  #  text = re.sub(r'[^\w\s@.,()\-:]', ' ', text)
     # --- Additional Spacing Fixes ---
    # Insert space between a lowercase letter and an uppercase letter (fix camelCase issues)
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)
    # Insert a space between letters and numbers if missing
    text = re.sub(r'(?<=[A-Za-z])(?=\d)', ' ', text)
    text = re.sub(r'(?<=\d)(?=[A-Za-z])', ' ', text)

    # Remove multiple spaces again if necessary
    #text = re.sub(r'\s+', ' ', text)
    text = fix_concatenated_words(text)
    for pattern, replacement in section_mapping.items():
        text = re.sub(pattern, f'\n{replacement}\n', text, flags=re.I)

    # Anonymize personal information





    # Remove empty lines and trim whitespace
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    return ' \n'.join(lines)
all_resumes_text_clean={}
for resume in resume_files:
   all_resumes_text_clean[resume]=clean_text(all_resumes_text[resume])


#Donnée structurée

Exraction de chaque section


In [ ]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

section_patterns = [
    [{"TEXT": "EXPERIENCE"}],
    [{"TEXT": "EDUCATION"}],
    [{"TEXT": "SKILLS"}],
    [{"TEXT": "PROJECTS"}],
    [{"TEXT": "CERTIFICATIONS"}],
    [{"TEXT": "AWARDS"}],
    [{"TEXT": "PUBLICATIONS"}],
    [{"TEXT": "VOLUNTEERING"}],
    [{"TEXT": "LANGUAGES"}],
    [{"TEXT": "INTERESTS"}],
    [{"TEXT": "REFERENCES"}]
]


matcher.add("RESUME_SECTIONS", section_patterns)

def extract_sections(text):
    doc = nlp(text)
    matches = matcher(doc)
    sections = {}
    last_end = 0

    # Process matches in document order
    for match_id, start, end in sorted(matches, key=lambda x: x[1]):
        section_name = doc[start:end].text.strip().upper()
        section_start = doc[start].idx  # Character start position
        section_end = doc[end].idx if end < len(doc) else len(text)

        # Capture content between previous section and current section
        if last_end > 0:
            prev_section_content = text[last_end:section_start].strip()
            if prev_section_content:
                sections.setdefault("UNCATEGORIZED", []).append(prev_section_content)

        # Capture section content
        content_start = section_end
        next_match = next((m for m in matches if m[1] > end), None)
        content_end = doc[next_match[1]].idx if next_match else len(text)
        if section_name not in sections:
         sections[section_name] = text[content_start:content_end].strip()

        else:
         sections[section_name] +="\n."+text[content_start:content_end].strip()
        last_end = content_end
    return sections


Extraction des compétences du section skills et LANGUAGES

---



In [ ]:
def extract_skills(sections):

    skills_section = sections.get("SKILLS", "")
    skills_section+=':'+sections.get("LANGUAGES", "")
    skills_section = re.sub(r'\n(?=\W)', ',', skills_section)
    # Common skill separators: commas, bullet points, semicolons
    skill_delimiters = r'[(.•●♦♦▪➢,)/:]+'
    skills_section = re.sub(r'\n', '', skills_section)

    # Split and clean skills
    skills = []
    if skills_section:
        # Split on delimiters and clean each skill
        raw_skills = re.split(skill_delimiters, skills_section)
        skills = [
            skill.strip().title()  # Title case for consistency
            for skill in raw_skills
            if skill.strip()  # Remove empty strings
        ]

    return skills
for resume in resume_files:
   extraction[resume]["skills"]=extract_skills(extract_sections(all_resumes_text_clean[resume]))
   extraction[resume]["cv_text"]=all_resumes_text_clean[resume]


extraction


{'/content/drive/MyDrive/resume/cv - Nes.pdf': {'name': 'Nesrine Khiyari',
  'email': 'nesrine.khiyari@etudiant-enit.utm.tn',
  'phone': '+216 56 42 57 04',
  'skills': ['And',
   'Time Management',
   'Adaptability',
   'Curiosity',
   'Comunication',
   'Data Analysis And Visualization',
   'Power Bi',
   'Advanced Excel',
   'Excel Solver',
   'Pandas',
   'Num Py',
   'Matplotlib And Skealearn',
   'Lean Tools',
   'Amdec And Swot',
   'Arabic',
   'Native French',
   'Advanced English',
   'Intermediate'],
  'cv_text': 'Nesrine Khiyari \nEmail : nesrine . khiyari @ etudiant - enit . utm . tn \nMobile : + 216 56 42 57 04 \nLinked In : khiyari - Nesrine \nIndustrial engineering student \nEDUCATION \nNational Engineering School of Tunis \nBachelor of Engineering ( B . E . ) in Industrial Engineering El Manar , Tunisia \nRelevant Coursework : Operations Management , Operations Research , Sept 2023 – Ongoing \nOptimization Techniques and Data mining . \nPreparatory Institute for Engine

Extraction d'éducation

In [ ]:
def extract_education_entities(sections):

    education_text=sections.get("EDUCATION", "")
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(education_text)


    education_entities = [(ent.text, ent.label_) for ent in doc.ents]
    return education_entities



entities = extract_education_entities((extract_sections(all_resumes_text_clean[0])))
print("Extracted Education Entities:", entities)

Extracted Education Entities: [('Bachelor of Science in Biochemistry', 'ORG'), ('New York University', 'ORG'), ('Computer Science \nNew York', 'ORG'), ('NY', 'GPE'), ('2021', 'CARDINAL')]


Extraction de l'expérience

In [ ]:
import re
from dateutil import parser
from datetime import datetime

def extract_experience_years(sections):
    experience_text = sections.get("EXPERIENCE", "")

    cleaned_lines = [
        re.sub(r'^[\s•\-\◦]+', '', line.strip())
        for line in experience_text.split('\n')
        if line.strip()
    ]

    entries = []
    total_days = 0  # Pour accumuler la durée totale

    for i, line in enumerate(cleaned_lines):
        doc = nlp(line)
        dates = [ent.text for ent in doc.ents if ent.label_ == "DATE"]

        if len(dates) >= 1:
            window_start = max(0, i - 2)
            window_end = min(len(cleaned_lines), i + 3)
            window_lines = cleaned_lines[window_start:window_end]
            combined_window = " ".join(window_lines)

            window_doc = nlp(combined_window)

            companies = [ent.text for ent in window_doc.ents if ent.label_ == "ORG"]
            company = companies[0] if companies else ""

            gpes = [ent.text for ent in window_doc.ents if ent.label_ == "GPE"]
            location = gpes[0] if gpes else ""

            # Calcul des dates
            start_date, end_date = None, None
            for d in dates:
                try:
                    parsed_date = parser.parse(d, fuzzy=True, default=datetime(1900, 1, 1))
                    if not start_date or parsed_date < start_date:
                        start_date = parsed_date
                    if not end_date or parsed_date > end_date:
                        end_date = parsed_date
                except:
                    continue

            if any("present" in d.lower() or "aujourd" in d.lower() for d in dates):
                end_date = datetime.now()

            duration_years = None
            if start_date and end_date:
                duration = end_date - start_date
                total_days += duration.days
                duration_years = round(duration.days / 365.25, 1)

            candidate_position = combined_window
            for token in (company, location, *dates):
                if token:
                    candidate_position = candidate_position.replace(token, "")
            candidate_position = candidate_position.strip(" ,.-")

            entry = {
                "dates": f"{start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')}" if start_date and end_date else ", ".join(dates),
                "company": company,
                "position": candidate_position,
                "location": location,
                "duration_years": duration_years
            }
            entries.append(entry)

    total_experience_years = round(total_days / 365, 1) if total_days else 0

    return {
        "entries": entries,
        "total_experience_years": total_experience_years
    }


In [ ]:
for resume in resume_files:
 extraction[resume]["years_experience"] =extract_experience_years(extract_sections(all_resumes_text_clean[resume]))["total_experience_years"]
extraction



{'/content/drive/MyDrive/resume/cv - Nes.pdf': {'name': 'Nesrine Khiyari',
  'email': 'nesrine.khiyari@etudiant-enit.utm.tn',
  'phone': '+216 56 42 57 04',
  'skills': ['And',
   'Time Management',
   'Adaptability',
   'Curiosity',
   'Comunication',
   'Data Analysis And Visualization',
   'Power Bi',
   'Advanced Excel',
   'Excel Solver',
   'Pandas',
   'Num Py',
   'Matplotlib And Skealearn',
   'Lean Tools',
   'Amdec And Swot',
   'Arabic',
   'Native French',
   'Advanced English',
   'Intermediate'],
  'cv_text': 'Nesrine Khiyari \nEmail : nesrine . khiyari @ etudiant - enit . utm . tn \nMobile : + 216 56 42 57 04 \nLinked In : khiyari - Nesrine \nIndustrial engineering student \nEDUCATION \nNational Engineering School of Tunis \nBachelor of Engineering ( B . E . ) in Industrial Engineering El Manar , Tunisia \nRelevant Coursework : Operations Management , Operations Research , Sept 2023 – Ongoing \nOptimization Techniques and Data mining . \nPreparatory Institute for Engine

In [ ]:
import json
with open('/content/drive/MyDrive/ResumeMatcher/resume_extract1.json', 'w') as f:
    json.dump(extraction, f)

print("💼 resume Saved!")

💼 resume Saved!
